In [30]:
# !pip3 install cx_Oracle --user

In [31]:
# OracleDB CX Oracle Python library 
# https://developer.oracle.com/dsl/prez-python-queries.html

In [32]:
import pandas as pd
import numpy as np
import cx_Oracle
from sqlalchemy import types, create_engine
from types import *
from pprint import pprint
import cx_Oracle
import sqlalchemy as sa
from datetime import datetime
import time

In [33]:
#All static vars go here
dbType = "OracleDB"

dbUser = "achem_drm"
dbPassword = "p1rosalma"
dbHost = "dalbcedd1.na.xom.com"
dbName = "bced"
dbPort = "55009"

#OracleDB Connection strings
oracleConnStr = 'oracle+cx_oracle://'+ dbUser + ':' + dbPassword + "@" + dbHost +':'+ dbPort +'/' + dbName
oracleDatabaseEngine = sa.create_engine(oracleConnStr)
oracleDbConnection = oracleDatabaseEngine.connect()

In [34]:
#Declare location of Config file and root directory for data here
#Assumes that these directories will be accessible to the code
configFile = "workspace/de/drm-de/excelUploadAutomation/config.xlsx"
configSheet = "config"
dataDirectory = "workspace/de/drm-de/excelUploadAutomation/sampleData/"

In [35]:
#Reading the config file here
configDf = pd.read_excel(open(configFile, 'rb'), sheet_name=configSheet, encoding='latin-1')
configDf = configDf.dropna(how='all')

In [36]:
def runOracleQuery(sqlQuery):
    ##################################################################################
    # Helper function
    # Function to run Oracle Query and print output
    # Takes input SQL Query and database congigurations
    # DB configuration is expected to be pre-declared in scope to the function call
    ##################################################################################
    try:
        oracleTns = cx_Oracle.makedsn(dbHost, dbPort, dbName)
        dbConnection = cx_Oracle.connect(dbUser, dbPassword, oracleTns)
        
        cursor = dbConnection.cursor()
        cursor.execute(sqlQuery)
        dbConnection.commit()

        try:
            data = cursor.fetchall()
            pprint(cursor.description)
            pprint(data)
        except:
            print("DEBUG: Not a data query, no data to print")

        cursor.close()
        dbConnection.close()
        print("DEBUG: Query executed")
        
    except Exception as e:
        print("ERROR: Something went wrong executing the query")
        print("ERROR: " + str(type(e).__name__))
        print("ERROR: " + str(e))   

In [37]:
def getTableAsDf(dbConnection, tableName):
    ##################################################################################
    # Helper function
    # Function to run Oracle Query and print output
    # Takes input SQL Query and database congigurations
    # No need to pass DB configuration if database variables are already declared where the function is called
    ##################################################################################
    query = "SELECT * FROM " + tableName
    try:
        readDf = pd.read_sql(query, con=dbConnection)
        return readDf
    except Exception as e:
        print("ERROR: Something went wrong while fetching the table")
        print("ERROR: " + str(type(e).__name__))
        print("ERROR: " + str(e))
        return pd.DataFrame()

In [38]:
def truncateTable(tableName, taskId):
    ##################################################################################
    # Helper function
    # Function to truncate a table from OracleDB
    # Takes input table name
    ##################################################################################
    try:
        sqlQuery = "TRUNCATE TABLE " + tableName
        runOracleQuery(sqlQuery)
        print("DEBUG: Truncated table " + tableName)
        return "Success"
    except Exception as e:
        print("ERROR: Something went wrong while truncating the table " + tableName + " in task id " + taskId)
        print("ERROR: " + str(type(e).__name__))
        print("ERROR: " + str(e))
        return "Failure"

In [39]:
def renameColums(df, columnsRenamed, taskId):
    ##################################################################################
    # Function to rename column names for a dataframe
    # Expects new names for EVERY column in the dataframe

    # columnsRenamed -> Comma seperated string containing new names for columns
    # taskId -> Task ID is passed to the function for debugging purpose
    # df -> Pandas dataframe to which the column rename function is applied 
    
    #Alternatively, to rename selective columns, enter oldName : newName key value pairs
    ##################################################################################

    if columnsRenamed.lower() in ["", None, "nan"]:
        return df

    columnsRenamed = columnsRenamed.split(',') 
    
    if len(columnsRenamed) == len(df.columns):
        columnsToRename = {}
        try:
            for i in range(len(columnsRenamed)):
                columnsToRename[df.columns.values[i].strip()] = columnsRenamed[i].strip()
            df = df.rename(columns=columnsToRename)
            return df

        except Exception as e:
            print("ERROR: Something went wrong while renaming columns for task id " + taskId)
            print("ERROR: " + str(type(e).__name__))
            print("ERROR: " + str(e))
            return pd.DataFrame()
    else:
        renameDict = {}
        for i in columnsRenamed:
            i = i.strip()
            i = i.split(':')
            renameDict[i[0].strip()] = i[1].strip()
        try:
            df.rename(columns = renameDict, inplace = True)
            return df
        except Exception as e:
            print("ERROR: Something went wrong while renaming columns for task id " + taskId)
            print("ERROR: " + str(type(e).__name__))
            print("ERROR: " + str(e))
            return pd.DataFrame()

In [40]:
def changeDataType(df, changeDataTypeConditions, taskId):
    ##################################################################################
    # Function to change data types for columns in a dataframe

    # changeDataTypeConditions -> Comma seperated key value pairs of format COLUMN-NAME: DATA-TYPE
    # taskId -> Task ID is passed to the function for debugging purpose
    # df -> Pandas dataframe to which the column data type conversion is applied
    
    # Supported Data types are:
    # object, int64, float64, datetime64, bool
    ##################################################################################
    
    if changeDataTypeConditions.lower() in ["", None, "nan"]:
        return df 
    
    changeDataTypeConditions = changeDataTypeConditions.split(',')
    changeDataTypeDictionary = {}
    
    try:
        for i in changeDataTypeConditions:
            i = i.strip()
            i = i.split(':')
            changeDataTypeDictionary[i[0].strip()] = i[1].strip()
            df = df.astype(changeDataTypeDictionary)
        return df
    
    except Exception as e:
        print("ERROR: Something went wrong while changing data type for task id " + taskId)
        print("ERROR: " + str(type(e).__name__))
        print("ERROR: " + str(e))
        print("DEBUG: Continuing without changing column type")
        return df

In [41]:
def filterData(df, filterConditionsRaw, taskId):
    ##################################################################################
    # Function to filter rows from data frame

    # filterDataConditions -> Expression for the filter to be applied
    # taskId -> Task ID is passed to the function for debugging purpose
    # df -> Pandas dataframe to which the filter is applied
    ##################################################################################
    
    try:
        
        filterDataConditions = filterConditionsRaw.split('||')
        filteredDf = pd.DataFrame()

        for condition in filterDataConditions:
            condition = condition.strip()
            condition = condition.split('&&')
            outDf = df

            for individualCondition in condition:
                individualCondition = individualCondition.strip()

                if '>' in individualCondition:
                    data = individualCondition.split('>')
                    colName = data[0].strip()
                    filterValue = data[1].strip()
                    if colName[0] == '"' and colName[-1] == '"':
                        colName = colName[1:-1]
                    if filterValue[0] == '"' and filterValue[-1] == '"':
                        filterValue = float(filterValue)
                    outDf = outDf[outDf[colName] > filterValue]


                elif '<'in individualCondition:
                    data = individualCondition.split('<')
                    colName = data[0].strip()
                    filterValue = data[1].strip()
                    if colName[0] == '"' and colName[-1] == '"':
                        colName = colName[1:-1]
                    if filterValue[0] == '"' and filterValue[-1] == '"':
                        filterValue = (filterValue[1:-1])
                    else:
                        filterValue = float(filterValue)
                    outDf = outDf[outDf[colName] < filterValue]

                elif '>=' in individualCondition:
                    data = individualCondition.split('>=')
                    colName = data[0].strip()
                    filterValue = data[1].strip()
                    if colName[0] =='"' and colName[-1] == '"':
                        colName = colName[1:-1]
                    if filterValue[0] == '"' and filterValue[-1] == '"':
                        filterValue = (filterValue[1:-1])
                    else:
                        filterValue = float(filterValue)
                    outDf = outDf[outDf[colName] >= filterValue]

                elif '<='in individualCondition:
                    data = individualCondition.split('<=')
                    colName = data[0].strip()
                    filterValue = data[1].strip()
                    if colName[0] == '"' and colName[-1] == '"':
                        colName = colName[1:-1]
                    if filterValue[0] == '"' and filterValue[-1] == '"':
                        filterValue = (filterValue[1:-1])
                    else:
                        filterValue = float(filterValue)
                    outDf = outDf[outDf[colName] <= filterValue]

                elif '==' in individualCondition:
                    data = individualCondition.split('==')
                    colName = data[0].strip()
                    filterValue = data[1].strip()
                    if colName[0] == '"' and colName[-1] == '"':
                        colName = colName[1:-1]
                    if filterValue[0] == '"' and filterValue[-1] == '"':
                        filterValue = (filterValue[1:-1])
                    else:
                        filterValue = float(filterValue)
                    outDf = outDf[outDf[colName] == filterValue]

                elif '!='in individualCondition:
                    data = individualCondition.split('!=')
                    colName = data[0].strip()
                    filterValue = data[1].strip()
                    if colName[0] == '"' and colName[-1] == '"':
                        colName = colName[1:-1]
                    if filterValue[0] == '"' and filterValue[-1] == '"':
                        filterValue = (filterValue[1:-1])
                    else:
                        filterValue = float(filterValue)
                    outDf = outDf[outDf[colName] != filterValue]

            filteredDf = pd.concat([filteredDf,outDf],ignore_index=True).drop_duplicates().reset_index(drop=True)

        return filteredDf

    except Exception as e:
        print("ERROR: Something went wrong while applying filter conditions for task id " + taskId)
        print("ERROR: " + str(type(e).__name__))
        print("ERROR: " + str(e))
        return df

In [42]:
def insertAdditionalColumns(df, insertColumns, taskId):
    ##################################################################################
    # Function to insert specific new columns into a dataframe

    # insertColumns -> Comma seperated list of new columns to be inserted
    # taskId -> Task ID is passed to the function for debugging purpose
    # df -> Pandas dataframe into which the new columns are inserted
    
    # Supported columns:
    # UPLOAD_TIME
    ##################################################################################
    
    if insertColumns.lower() in ["", None, "nan"]:
        return df 

    insertColumns = insertColumns.split(',')
    
    for i in insertColumns:
        if i.strip().lower() == "upload_time":
            now = datetime.now()
            current_time = now.strftime("%H:%M:%S")
            df['UPLOAD_TIME'] = (current_time)
            
        else:
            continue
            
    return df

In [43]:
def replaceData(df, replaceDataConditions, taskId):
    ##################################################################################
    # Function to replace values in a dataframe

    # replaceDataConditions -> Key Value pair where key is the column name, and value is a doctionary with data replacement conditions
    # taskId -> Task ID is passed to the function for debugging purpose
    # df -> Pandas dataframe to which the data replacements are applied
    ##################################################################################
    
    if replaceDataConditions.lower() in ["", None, "nan"]:
        return df 
    
    replaceDataConditions = replaceDataConditions.split(',')
    
    try:
        for i in replaceDataConditions:
            i = i.strip()
            i = i.split(':',1)
            col_name=i[0]
            conditions=i[1].replace('{','').replace('}',"").split('|')
            col_type=str(df[col_name].dtype)
            for condition in conditions:
                condition = condition.strip()
                old,new = condition.split(":")
                if old.lower() in ['nan','n.a.','n.a','na','']:
                    if 'float' in col_type:
                        df[col_name]=df[col_name].fillna(float(new))
                    elif 'int' in col_type:
                        df[col_name]=df[col_name].fillna(int(new))
                    elif 'obj' in col_type:
                        df[col_name]=df[col_name].replace(old,new)
                else:
                    if 'float' in col_type:
                        df[col_name]=df[col_name].replace(float(old),float(new))
                    elif 'int' in col_type:
                        df[col_name]=df[col_name].replace(int(old),int(new))
                    elif 'obj' in col_type:
                        df[col_name]=df[col_name].replace(old,new)
        return df
    except Exception as e:
        print("ERROR: Something went wrong while replacing data for task id " + taskId)
        print("ERROR: " + str(type(e).__name__))
        print("ERROR: " + str(e))
        return pd.DataFrame()


In [44]:
def formatDateTime(df, formatDateTimeConditions, taskId):
    ##################################################################################
    # Function to format dates in a dataframe

    # formatDateTimeConditions -> Comma seperated key value pair where key is column name and value is new date format
    # taskId -> Task ID is passed to the function for debugging purpose
    # df -> Pandas dataframe to which the date transformations are applied
    ##################################################################################
    
    if formatDateTimeConditions.lower() in ["", None, "nan"]:
        return df 
    
    try:
        formatDateTimeConditions=formatDateTimeConditions.split(',')
        for i in formatDateTimeConditions:
            i = i.strip()
            col,new_format = i.split(':',1)
            df[col]=pd.to_datetime(df[col]).dt.strftime(new_format)
        return df
    except Exception as e:
        print("ERROR: Something went wrong while formating datetime for task id " + taskId)
        print("ERROR: " + str(type(e).__name__))
        print("ERROR: " + str(e))
        return pd.DataFrame()


In [45]:
def dfInsertToTable(df, tableName, taskId, startRow = 0):
                
    startRow = int(startRow)

    oracleTns = cx_Oracle.makedsn(dbHost, dbPort, dbName)
    dbConnection = cx_Oracle.connect(dbUser, dbPassword, oracleTns, encoding = "UTF-8", nencoding = "UTF-8")
    
    cursor = dbConnection.cursor()
    cursor1 = dbConnection.cursor()
    
    cursor1.execute("SELECT * FROM " + tableName + " fetch first 3 rows only")
    
    dbConnection.commit()
    rowsTemp = cursor1.fetchall()
    cursor.bindarraysize = len(rowsTemp)
    db_types = (d[1] for d in cursor1.description)
    cursor.setinputsizes(*db_types)
    cursor1.close()
            
    trackerIndex = 0
    for index, row in df[startRow:].iterrows():
        try:
            trackerIndex = index

            rowData = ""
            counter = 1
            for i in row:
                rowData = rowData + " :" + str(counter) + " ,"
                counter += 1
            rowData = rowData[:-2]

            columnNamesString = []
            for i in df.columns.values:
                temp = '"' + i + '"'
                columnNamesString.append(temp)
            columns = (',').join(columnNamesString)

            sqlQuery = "INSERT INTO " + tableName + " (" + columns +  ") VALUES" + "(" + rowData + ")"
            
            map(lambda x: x.encode('utf-8'), row)
            
            cursor.execute(sqlQuery, row)
            dbConnection.commit()
            
            if index % 1000 == 0:
                print("DEBUG: Writing row " + str(index) + " for task id " + str(taskId))
        
        except Exception as e:
            #Failed at row startRow
            print(row)
            print("ERROR: Something went wrong while writing to table " + tableName + " for task id " + taskId)
            print("DEBUG: Failed at row " + str(trackerIndex))
            print("ERROR: " + str(type(e).__name__))
            print("ERROR: " + str(e))
            cursor.close()
            dbConnection.close()
            return "Failure"
        
    print("DEBUG: Insert successful")
    cursor.close()
    dbConnection.close()
    return "Success"

In [46]:
def dfUpdateToTable(df, tableName, taskId, upsertKey, startRow = 0):
    startRow = int(startRow)
    
    upsertKeys = upsertKey.strip().split(',')
    map(lambda x: x.strip(), upsertKeys)
    
    oracleTns = cx_Oracle.makedsn(dbHost, dbPort, dbName)
    dbConnection = cx_Oracle.connect(dbUser, dbPassword, oracleTns, encoding = "UTF-8", nencoding = "UTF-8")
        
    
    
    cursor = dbConnection.cursor()
    ############################Delete query creation#########################
    try:
        tempDf = df[upsertKeys].drop_duplicates() 
        
        
        for index, row in tempDf.iterrows():
            whereClause = ""
            counter = 1

            for i in upsertKeys:
                whereClause = whereClause +  '"' + i.strip() + '" = :' + str(counter) + ' AND '
                counter += 1
            
            whereClause = whereClause[:-4]
            sqlQueryDelete = "DELETE FROM " + str(tableName) + " WHERE " + whereClause
            map(lambda x: x.encode('utf-8'), row)
            cursor.execute(sqlQueryDelete, row)
            dbConnection.commit()
            
        print("DEBUG: Deleted the existing values, Now starting insert")
    except Exception as e:
        print("ERROR: Something went wrong while deleting for upsert keys in table " + tableName + " for task id " + taskId)
        print("ERROR: " + str(type(e).__name__))
        print("ERROR: " + str(e))
    cursor.close()
    ############################Delete query creation#########################
    
    
    ############################Insert query creation#########################
    
    cursor1 = dbConnection.cursor()
    cursor = dbConnection.cursor()

    cursor1.execute("SELECT * FROM " + tableName + " fetch first 3 rows only")
    dbConnection.commit()
    rowsTemp = cursor1.fetchall()
    cursor.bindarraysize = len(rowsTemp)
    db_types = (d[1] for d in cursor1.description)
    cursor.setinputsizes(*db_types)
    cursor1.close()
            
    trackerIndex = 0
    for index, row in df[startRow:].iterrows():
        try:
            trackerIndex = index
            
            rowData = ""
            counter = 1
            for i in row:
                rowData = rowData + " :" + str(counter) + " ,"
                counter += 1
            rowData = rowData[:-2]
            columnNamesString = []
            for i in df.columns.values:
                temp = '"' + i + '"'
                columnNamesString.append(temp)
            columns = (',').join(columnNamesString)
            sqlQueryInsert = "INSERT INTO " + tableName + " (" + columns +  ") VALUES" + "(" + rowData + ")"
            map(lambda x: x.encode('utf-8'), row)
            cursor.execute(sqlQueryInsert, row)
            dbConnection.commit()
            
            if index % 1000 == 0:
                print("DEBUG: Writing row " + str(index) + " for task id " + str(taskId))
            
        except Exception as e:
            #Failed at row startRow
            print("ERROR: Something went wrong while writing to table " + tableName + " for task id " + taskId)
            print("DEBUG: Failed at row " + str(trackerIndex))
            print("ERROR: " + str(type(e).__name__))
            print("ERROR: " + str(e))
            cursor1.close()
            cursor.close()
            dbConnection.close()
            return "Failure"
    ############################Insert query creation#########################
        
    cursor.close()  
    dbConnection.close() 
    print("DEBUG: Update successful")
    return "Success"

In [47]:
def writeDfToDb(df, tableName, upsertKey, truncateBool, resumeFromRow, taskId):
    
    if dbType == "OracleDB":

        if resumeFromRow not in ["", "0", 0, "nan", None]:
            #If resuming, then no need to truncate
            
            if upsertKey in ["", "nan",None]:
                #Not upserting, simply insert data
                dfInsertToTable(df, tableName, taskId, resumeFromRow)
            else:
                #Upserting / Updating the data for a particular key
                dfUpdateToTable(df, tableName, taskId, upsertKey, resumeFromRow)
                
        else:
            if truncateBool in ["1", 1, True, "True"]:
                deleteStatus = truncateTable(tableName, taskId)
                if deleteStatus == "Failure":
                    return "Failure"
            
            if upsertKey in ["", "nan",None]:
                #Not upserting, simply insert data
                dfInsertToTable(df, tableName, taskId)
                
            else:
                #Upserting / Updating the data for a particular key
                dfUpdateToTable(df, tableName, taskId,  upsertKey)

In [48]:
for index, row in configDf.iterrows():
    #Iterate the config for individual data transformations
    
    #Record the start time for debuging purpose
    start_time = time.time()

    ####################################################################################################
    #Reading configurations data here
    
    #Adding '1' to ignore column will skip the test
    if row['ignore'] in ['1', 1]:
        continue
    
    taskId = row['id']
    if taskId == "":
        continue
    
        
    dataFile = dataDirectory + str(row['fileName']).strip()
    sheetName = str(row['sheetName']).strip()
    
    skipFromRow = row['skipFromRow']
    skipTillRow = row['skipTillRow']
    headerRow = row['headerRow']
    
    columns = str(row['columns']).strip()
    
    columnsRenamed = str(row['columnsRenamed']).strip()
    filterDataConditions = str(row["filterDataCondition"]).strip()
    replaceDataConditions = str(row["replaceDataCondition"]).strip()
    changeDataTypeConditions = str(row["changeDataTypeCondition"]).strip()
    formatDateTimeConditions = str(row["formatDateTimeCondition"]).strip()

    tableName = str(row['tableName']).strip()
    
    truncateBool = str(row['truncateBool']).strip()
    upsertKey = str(row['upsertKey']).strip()
        
    #Insert Additional columns
    additionalCols = str(row['additionalCols']).strip()
    ####################################################################################################
     

        
        
        
    ####################################################################################################
    #Making sanity checks on basic data
    
    print()
    print("DEBUG: Running task " + str(taskId) + " for table " + str(tableName))
    
    if type(taskId)!=int:
        print("ERROR: 'Task id' should be integer. Incorrect format for task id " + taskId)
        continue
    else:
        taskId = str(taskId)
       
    try:
        headerRow = int(row['headerRow'])
    except Exception as e:
        headerRow = 1
        
    try:
        skipFromRow = int(row['skipFromRow']) - 1
    except Exception as e:
        skipFromRow = 0
        
    try:
        skipTillRow = int(row['skipTillRow']) - 1
    except Exception as e:
        skipTillRow = 0
        
    if truncateBool not in ["", "0", "1", "nan"]:
        print("ERROR: Invalid truncate Bool, please check the configuration for task id " + taskId)
        continue
                
    if columns in ["", None, "nan"]:
        columns = None
    
    if str(row['resumeFromRow']) in ["", "nan", 0, None, "0", "Nan", "NaN"]:
        resumeFromRow = 0
    else:
        #Restart processing from row (Failure handling)
        resumeFromRow = int(row['resumeFromRow'])
        
    ####################################################################################################
    
    
    
    #Read the excel file to be uploaded
    dataDf = pd.read_excel(open(dataFile, 'rb'), sheet_name=sheetName ,  usecols=columns, skiprows=range(skipFromRow, skipTillRow))
    
    ####################################################################################################
    #Applying data transformations
    
    
    #Rename columns using input list of format: old1:new1, old2:new2......
    dataDf = renameColums(dataDf, columnsRenamed, taskId)
    if (dataDf.empty == True):
        continue
    
    #Apply filters on excel data using input conditions of format: col1<10 & col2>100 | col3=100 | col4!=19
    dataDf = filterData(dataDf, filterDataConditions, taskId)
    if (dataDf.empty == True):
        continue
        
        
    dataDf = dataDf.replace('#', np.nan)
    dataDf = dataDf.replace(np.nan, '', regex=True)
    dataDf = dataDf.fillna()
    
    #Replace data for selected columns based on input conditions of format: col5:{25:26}, col6:{123:124}, col7:{nan:0}    
    dataDf = replaceData(dataDf, replaceDataConditions, taskId)
    if (dataDf.empty == True):
        continue    
    
    #Format DateTime columns based on input conditions of format: col8:%Y-%b-%d, col9:%Y-%b-%d %H:%M:%S %p
    dataDf = formatDateTime(dataDf, formatDateTimeConditions, taskId)
    if (dataDf.empty == True):
        continue 
        
        
    #Insert additional columns 
    dataDf = insertAdditionalColumns(dataDf, additionalCols, taskId)
    if (dataDf.empty == True):
        continue 
        
    
    #Change data type for selected columns based on input conditions of format: col5:float, col6:int, col7:str
    dataDf = changeDataType(dataDf, changeDataTypeConditions, taskId)
    if (dataDf.empty == True):
        continue 
        
        
    ####################################################################################################
    
    try:
        writeDfToDb(dataDf, tableName, upsertKey, truncateBool, resumeFromRow, taskId)
        print("DEBUG: Time for execution --- %s seconds ---" % (time.time() - start_time))
        
    except Exception as e:
        print("ERROR: Something went wrong in non database operations while writing for task id " + taskId)
        print("ERROR: " + str(type(e).__name__))
        print("ERROR: " + str(e))


DEBUG: Running task 17 for table DP_ZEMA_IHS_DAILY_BKUP


ValueError: Must specify a fill 'value' or 'method'.

In [20]:
# temp = getTableAsDf(oracleDbConnection, "EXCEL_UPLOAD_MACH1_MASTER_TST4")
# print(temp.shape)
# print(temp.columns)
# query = """
# DELETE FROM EXCEL_UPLOAD_MACH1_MASTER_TST4 WHERE 'IPC Month' = '01/2018'
# """
# runOracleQuery(query)

In [50]:
# query = """
# SELECT  * FROM  DP_MACH1_FINAL_TABLE_NEW fetch first 3 rows only
  
# """
# runOracleQuery(query)


# PRODUCT : str, Product Code: str, GEOGRAPHY:str, CONCEPT:str, GRADE: str, TERMS:str, Price ID:int, UNIT: str, PRICE :int

In [22]:
# df = getTableAsDf(oracleDbConnection, "DP_ZEMA_IHS_DAILY_BKUP")
# print(df.dtypes)


In [23]:
# query = """
# TRUNCATE TABLE EXCEL_UPLOAD_MACH1_MASTER_TST4  
# """
# runOracleQuery(query)
# 
# query = """CREATE TABLE "EXCEL_UPLOAD_ICIS_INDEX_TEST"     
# ( 
# "MONTH" VARCHAR2(255), 
# "Metric Name" VARCHAR2(255), 
# "PRODUCT" VARCHAR2(255), 
# "VALUE" NUMBER(10,5), 
# "REGION" VARCHAR2(255), 
# "CURR" VARCHAR2(255), 
# "UPLOAD_TIME" VARCHAR2(255)    
# )"""
# runOracleQuery(query)

In [24]:
# query = """
# select * from EXCEL_UPLOAD_ICIS_INDEX_TEST  
# """
# runOracleQuery(query)

# Create new table
# query = """
# CREATE TABLE Excel_Upload_automation_JayFan 
# (col1 int,
# col2 int,
# col3 int,
# col4 int,
# col5 float,
# col6 float,
# col7 VARCHAR(26),
# col8 VARCHAR(26),
# col9 VARCHAR(26))
# """

# query = """
# SELECT * FROM Excel_Upload_automation_JayFan
# """


# query = """
# CREATE TABLE Excel_Upload_automation_JF1 
# (col1 varchar(26), pm varchar(26))
# """
# runOracleQuery(query)


# query = """
# TRUNCATE TABLE Excel_Upload_automation_JF1 
# """
# runOracleQuery(query)

# query = """
# INSERT INTO Excel_Upload_automation_JF1 (col1)
# VALUES (100)
# """
# runOracleQuery(query)



# print(getTableAsDf(oracleDbConnection, "EXCEL_UPLOAD_MACH1_MASTER_TEST"))

# query = """
# SELECT table_name FROM all_tables
# """
# runOracleQuery(query)



# query = """No Title
# CREATE TABLE "ACHEM_DRM"."EXCEL_UPLOAD_MACH1_MASTER_TEST"     
# ( 
# 			"BU" VARCHAR2(50), 
# 		  "IPC Month" VARCHAR2(100), 
# 		  "Shipto ID" VARCHAR2(100), 
# 		  "Plant ID" VARCHAR2(50), 
# 		  "Material ID" VARCHAR2(100), 
# 		  "INCOTERMS" VARCHAR2(100), 
# 		  "Ship Condition ID" VARCHAR2(100), 
# 		  "Mode of Trans" VARCHAR2(100), 
# 		  "Sales type" VARCHAR2(100), 
# 		  "Sales Order Type" VARCHAR2(100), 
# 		  "End use ID" VARCHAR2(100), 
# 		  "VOLUME" NUMBER(15,5), 
# 		  "GR" NUMBER(15,5), 
# 		  "Cash Disc" NUMBER(15,5), 
# 		  "SURCHARGES" NUMBER(15,5), 
# 		  "Freight Revenue" NUMBER(15,5), 
# 		  "EPD" NUMBER(15,5), 
# 		  "NR" NUMBER(15,5), 
# 		  "Vol Rebt" NUMBER(15,5), 
# 		  "NNR" NUMBER(15,5), 
# 		  "NRM" NUMBER(15,5), 
# 		  "Import Duties" NUMBER(15,5), 
# 		  "Fees Oth Than Duty" NUMBER(15,5), 
# 		  "Duties Fees and Insur" NUMBER(15,5), 
# 		  "NRM & Packaging" NUMBER(15,5), 
# 		  "COGS" NUMBER(15,5), 
# 		  "Freight (STO)" NUMBER(15,5), 
# 		  "Inbound Expenses" NUMBER(15,5), 
# 		  "Gross Margin" NUMBER(15,5), 
# 		  "Chems & Cats" NUMBER(15,5), 
# 		  "Fuels & Util" NUMBER(15,5), 
# 		  "Other Variable Selling" NUMBER(15,5), 
# 		  "Freight - CGL8" NUMBER(15,5), 
# 		  "Freight - CGL7" NUMBER(15,5), 
# 		  "VSC" NUMBER(15,5), 
# 		  "Variable Distri" NUMBER(15,5), 
# 		  "VM" NUMBER(15,5), 
# 		  "Cust/Sales Area" VARCHAR2(100), 
# 		  "Sales Organization" VARCHAR2(50), 
# 		  "Mfg Plant ID" VARCHAR2(50), 
# 		  "UPLOAD_TIME" DATE    ) 
# """
# runOracleQuery(query)

# query = """CREATE TABLE "EXCEL_UPLOAD_ICIS_INDEX_TEST"     
# ( 
# 			"MONTH" VARCHAR2(255), 
# 		  "Metric Name" VARCHAR2(255), 
# 		  "PRODUCT" VARCHAR2(255), 
# 		  "REGION" VARCHAR2(255), 
# 		  "VALUE" NUMBER(10,5), 
# 		  "CURR" VARCHAR2(255), 
# 		  "UPLOAD_TIME" VARCHAR2(255)    )"""
# runOracleQuery(query)


# query = """
# drop table EXCEL_UPLOAD_ICIS_INDEX_TEST  
# """
# runOracleQuery(query)


# query = """CREATE TABLE "EXCEL_UPLOAD_MACH1_SHIPTO_TST3"     
# ( 
# 			"Cust/Sales Area" VARCHAR2(50), 
# 		  "Sales Organization" VARCHAR2(50), 
# 		  "BU" VARCHAR2(50), 
# 		  "Shipto ID" VARCHAR2(50), 
# 		  "Soldto ID" VARCHAR2(50), 
# 		  "Soldto Name" VARCHAR2(65), 
# 		  "Parent ID" VARCHAR2(50), 
# 		  "Parent Name" VARCHAR2(50), 
# 		  "Sales Manager ID" VARCHAR2(50), 
# 		  "Sales Manager" VARCHAR2(50), 
# 		  "Sales Person ID" VARCHAR2(50), 
# 		  "Sales Person Name" VARCHAR2(61), 
# 		  "Cust.Serv.Cl.of Sold to" VARCHAR2(50), 
# 		  "Cust.Serv.Cl.ShipTo" VARCHAR2(50), 
# 		  "Cust Class Name" VARCHAR2(50), 
# 		  "Alt Cust Grp3" VARCHAR2(50), 
# 		  "Alt Cust Grp2" VARCHAR2(65), 
# 		  "Alt Cust Grp1" VARCHAR2(50), 
# 		  "AREA" VARCHAR2(50), 
# 		  "Area Name" VARCHAR2(50), 
# 		  "REGION" VARCHAR2(50), 
# 		  "Region Name" VARCHAR2(50), 
# 		  "SUPERREGION" VARCHAR2(50), 
# 		  "SuperRegion Name" VARCHAR2(50), 
# 		  "Ship to Country ID" VARCHAR2(50), 
# 		  "Ship to Country Name" VARCHAR2(50), 
# 		  "VOLUME" VARCHAR2(50), 
# 		  "LOCATION" VARCHAR2(50), 
# 		  "Country Sub Div" VARCHAR2(50), 
# 		  "Country Sub Div Code" VARCHAR2(10), 
# 		  "UPLOAD_TIME" VARCHAR2(30)    ) 
# """
# runOracleQuery(query)


# query = """
# CREATE TABLE  "EXCEL_UPLOAD_MACH1_MASTER_TST4"     
# ( 
# 			"BU" VARCHAR2(50), 
# 		  "IPC Month" VARCHAR2(100), 
# 		  "Shipto ID" VARCHAR2(100), 
# 		  "Plant ID" VARCHAR2(50), 
# 		  "Material ID" VARCHAR2(100), 
# 		  "INCOTERMS" VARCHAR2(100), 
# 		  "Ship Condition ID" VARCHAR2(100), 
# 		  "Mode of Trans" VARCHAR2(100), 
# 		  "Sales type" VARCHAR2(100), 
# 		  "Sales Order Type" VARCHAR2(100), 
# 		  "End use ID" VARCHAR2(100), 
# 		  "VOLUME" NUMBER(15,5), 
# 		  "GR" NUMBER(15,5), 
# 		  "Cash Disc" NUMBER(15,5), 
# 		  "SURCHARGES" NUMBER(15,5), 
# 		  "Freight Revenue" NUMBER(15,5), 
# 		  "EPD" NUMBER(15,5), 
# 		  "NR" NUMBER(15,5), 
# 		  "Vol Rebt" NUMBER(15,5), 
# 		  "NNR" NUMBER(15,5), 
# 		  "NRM" NUMBER(15,5), 
# 		  "Import Duties" NUMBER(15,5), 
# 		  "Fees Oth Than Duty" NUMBER(15,5), 
# 		  "Duties Fees and Insur" NUMBER(15,5), 
# 		  "NRM & Packaging" NUMBER(15,5), 
# 		  "COGS" NUMBER(15,5), 
# 		  "Freight (STO)" NUMBER(15,5), 
# 		  "Inbound Expenses" NUMBER(15,5), 
# 		  "Gross Margin" NUMBER(15,5), 
# 		  "Chems & Cats" NUMBER(15,5), 
# 		  "Fuels & Util" NUMBER(15,5), 
# 		  "Other Variable Selling" NUMBER(15,5), 
# 		  "Freight - CGL8" NUMBER(15,5), 
# 		  "Freight - CGL7" NUMBER(15,5), 
# 		  "VSC" NUMBER(15,5), 
# 		  "Variable Distri" NUMBER(15,5), 
# 		  "VM" NUMBER(15,5), 
# 		  "Cust/Sales Area" VARCHAR2(100), 
# 		  "Sales Organization" VARCHAR2(50), 
# 		  "Mfg Plant ID" VARCHAR2(50), 
# 		  "UPLOAD_TIME" DATE    ) 
# """
# runOracleQuery(query)

